In [1]:
import pandas as pd
import numpy as np
import pickle
from tqdm.notebook import tqdm

from utils.Sharp import SharpPortfolio
from utils.Markov import MarkovPortfolio
from utils.Tobin import TobinPortfolio

# Disable jedi autocompleter
%config Completer.use_jedi = False

In [2]:
df = pd.read_csv('results/prediction_table.csv', index_col=0)
pct = pd.read_csv('data/ticker_data_preprocessed.csv', index_col=0)
pct = pct.drop(['sector', '2022-07-29', '2022-07-28', '2022-07-27', '2022-07-26', '2022-07-25'], axis=1)
sp500 = pd.read_csv('data/sp500.csv', index_col=0)['Изм. %'][5:]
bonds = pd.read_csv('data/bonds.csv', index_col=0)['Изм. %'][5:]

In [3]:
sp500 = sp500.apply(lambda x: float(x[:-1].replace(',', '.')) / 100)
bonds = bonds.apply(lambda x: float(x[:-1].replace(',', '.')) / 100)

# Обработка даты 

def month(dates : list):
    new_dates = []
    for d in dates:
        date = d.split()
        months = {'янв.' : '01', 'февр.': '02', 'мар.': '03', 'апр.': '04', 'мая': '05',
                  'июн.' : '06', 'июл.': '07', 'авг.': '08', 'сент.': '09', 'окт.': '10', 'нояб.': '11', 'дек.': '12'}
        date = date[0]+'-'+months[date[1]]+'-'+date[2]
        new_dates.append(date)
    return new_dates

sp500.index = month(sp500.index)

## Создаю словари для каждого метода кластеризации

In [4]:
cols = df.columns[1:]

**общий словарь**: ключ - метод кластеризации, значение - словарь(*ключ - номер кластера, значения - тикеры*)

In [5]:
all_accets = {}
for col in cols:
    accets = {}
    for i in set(df[col].unique()):
        accets[i] = list(df.loc[df[col] == i].index.values)
    all_accets[col] = accets

## Выберу из каждого кластера 2 лучшие акции по значению Шарпа и создам портфель для каждого метода кластеризации

In [6]:
def choose_best_accets(dict_accets, df_pct):
    accets_to_port = []
    for cluster in dict_accets.values():
        if len(cluster)==1:
            accets_to_port.append(cluster[0])
        else:
            dict_with_sharp = {}
            sharp = 0
            first_sharp = -100
            second_sharp = -200
            # считаем Шарпа для каждой акции
            for ticker in cluster:
                tickers_pct = df_pct.loc[df_pct.index == ticker].values
                sharp = (tickers_pct.mean()-0.03)/tickers_pct.std()
                dict_with_sharp[sharp] = ticker

            # выбираем 2 лучших акции
            for sharp in dict_with_sharp.keys():
                if first_sharp < sharp:
                    second_sharp = first_sharp
                    first_sharp = sharp
                elif first_sharp > sharp and second_sharp < sharp:
                    second_sharp = sharp
            accets_to_port.append(dict_with_sharp[first_sharp])
            accets_to_port.append(dict_with_sharp[second_sharp])
    return accets_to_port    

In [7]:
# Сохраняю расчеты в словарь: ключ - название портфеля(метод кластеризации), значение - тикеры
ports = {}
for col in cols:
    ports[col] = choose_best_accets(all_accets[col], pct)

## Найду лучшие методы кластеризации исходя из доходности

Веса в портфеле определяются значением коэффициента Шарпа

In [8]:
bonds.index = pd.to_datetime(bonds.index)
sp500.index = pd.to_datetime(sp500.index)

# Настройка размерности

sp500 = sp500.drop(list(set(sp500.index) - set(bonds.index)))
bonds = bonds.drop(list(set(bonds.index) - set(sp500.index)))

In [9]:
pct_monthly = {}
sp500_monthly = {}
bonds_monthly = {}

pct = pct.T[:-1]
pct.index = pd.to_datetime(pct.index)

In [10]:
# Настройка размерности

pct = pct.drop(list(set(pct.index) - set(bonds.index)))
bonds = bonds.drop(list(set(bonds.index) - set(pct.index)))
sp500 = sp500.drop(list(set(sp500.index) - set(pct.index)))

In [11]:
dates = list((sp500.reset_index().groupby(pd.Grouper(key="index", freq="M")).std()).index)

In [12]:
last_date = dates[0]
pct_monthly[last_date] = pct[(pct.index<=last_date)]
sp500_monthly[last_date] = sp500[(sp500.index<=last_date)]
bonds_monthly[last_date] = bonds[(bonds.index<=last_date)]
for date in dates[1:]:
    pct_monthly[date] = pct[((pct.index>last_date)&(pct.index<=date))]
    sp500_monthly[date] = sp500[((sp500.index>last_date)&(sp500.index<=date))]
    bonds_monthly[date] = bonds[((bonds.index>last_date)&(bonds.index<=date))]
    last_date=date

Для каждого портфеля создаю словарь с доходностями

In [13]:
def create_dict_monthly(dates, pct_monthly, tickers):
    d = {}
    for date in dates:
        d[date] = pct_monthly[date][tickers]
    return d

In [14]:
def create_dict_half_year(dates, pct_monthly, tickers):
    d = {}
    for date in range(6, len(dates)):
        d[dates[date]] = pd.concat([pct_monthly[dates[date-1]][tickers], 
                                   pct_monthly[dates[date-2]][tickers],
                                   pct_monthly[dates[date-3]][tickers], 
                                   pct_monthly[dates[date-4]][tickers],
                                   pct_monthly[dates[date-5]][tickers],
                                   pct_monthly[dates[date-6]][tickers]])
    return d

In [15]:
port_names = list(ports.keys())
portfolios = {}
for port in port_names:
    portfolios[port] = create_dict_monthly(dates, pct_monthly, ports[port])

In [16]:
pct_half_year = {}
for port in port_names:
    pct_half_year[port] = create_dict_half_year(dates, pct_monthly, ports[port])

In [17]:
dates[6]

Timestamp('2018-07-31 00:00:00', freq='M')

In [18]:
pct_half_year['original_n'][dates[6]]

,FCX,MOS,TWTR,MTCH,CZR,PENN,NWL,SYY,APA,OXY,...,ALGN,DXCM,AAL,UAL,SPG,VTR,ENPH,SEDG,AES,NRG
2018-06-04,0.005257,-0.007231,0.033561,-0.005636,-0.007709,-0.003252,0.004325,0.005045,0.005389,0.003055,...,0.030352,0.013666,0.019545,0.010542,0.011318,0.002735,0.001730,0.023766,0.010946,-0.001769
2018-06-06,0.031162,0.043416,0.007538,-0.004441,0.019101,0.021759,0.022053,-0.000760,0.008408,0.011110,...,0.000838,-0.002542,0.000226,0.015003,0.014362,-0.002936,-0.056939,0.004695,-0.032483,0.008618
2018-06-07,-0.014835,0.007565,-0.009975,-0.040385,-0.025358,-0.002366,0.015353,-0.000304,0.054826,0.000231,...,-0.020354,-0.049899,-0.007920,-0.011654,-0.002904,0.004049,0.003774,-0.000935,0.016787,-0.012964
2018-06-08,0.005577,0.005461,0.038035,0.015904,-0.016968,-0.039431,0.008582,0.005931,0.011737,-0.011679,...,0.021661,0.039222,-0.016651,0.006471,0.003580,0.005132,-0.054511,0.008419,0.003931,-0.010448
2018-06-11,0.008874,-0.000679,0.005096,0.013488,-0.002302,-0.003086,0.036872,0.003930,0.055871,0.007956,...,0.005106,0.021613,0.024356,0.030862,-0.001693,0.001276,-0.081511,-0.018553,0.004698,-0.014781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-01-25,0.011224,-0.018750,-0.009388,0.013868,-0.014948,0.016005,-0.205572,0.004080,-0.013923,-0.010572,...,0.004701,0.018705,-0.031758,-0.043012,-0.011439,0.009304,0.034653,0.031162,0.029570,0.014679
2018-01-26,-0.014127,0.034845,0.095217,0.021621,0.003035,-0.011209,0.020959,0.002032,0.013277,0.005734,...,0.038054,-0.012581,0.000377,0.007264,-0.002984,-0.009218,0.023924,0.009615,0.020017,-0.009042
2018-01-29,0.004606,0.002896,0.037495,0.012093,-0.031770,-0.011642,0.001184,-0.011385,-0.029742,-0.012179,...,-0.025355,-0.007576,-0.007349,-0.011719,-0.004826,-0.006441,-0.051402,0.009524,0.007679,-0.019708
2018-01-30,-0.025471,-0.018412,0.017474,-0.011949,0.007812,-0.007750,0.014590,-0.001420,-0.044587,-0.019412,...,-0.024172,0.028973,-0.001708,0.018699,-0.001903,-0.015847,0.000000,-0.039083,-0.016935,-0.028667


В переменной portfolios хранятся словари(каждый метод кластеризации) с доходностями своих тикеров помесячно    
В переменной pct_half_year хранятся словари(каждый метод кластеризации) с доходностями своих тикеров за полгода

### Расчет портфелей

1) Выбираем метод кластеризацци
2) Считаем для него оптимизацию

In [19]:
def calc_ports(test, riskless_ret, index, mu, sigma):
    sharp_values, markov_values, tobin_values, traynor_values, qsharp_values = [], [], [], [], []
    # Расчет Шарпа
    sharp = SharpPortfolio(mu, sigma, riskless_ret.mean())
    sh_values = test @ sharp.fit()
    for value in sh_values:
        sharp_values.append(value)

    # Расчет Марковица
    markov = MarkovPortfolio(mu, sigma, index)
    m_values = test @ markov.fit()
    for value in m_values:
        markov_values.append(value)

    # Расчет Тобина

    tobin = TobinPortfolio(mu, sigma, riskless_ret.mean(), index.std())
    tob_values = test @ tobin.fit()
    for value in tob_values:
        tobin_values.append(value)


    df_all_ports = pd.DataFrame({'date':test.index,
                                 'sharp':sharp_values,
                                 'markov':markov_values,
                                 'tobin':tobin_values})
    return df_all_ports

In [20]:
dict_ports = {}
for port in tqdm(list(portfolios.keys())):
    cluster = portfolios[port]
    test = cluster[dates[6]]
    riskless_ret = bonds_monthly[dates[6]]
    index = sp500_monthly[dates[6]]
    
    half_year = pct_half_year[port][dates[6]]
    mu = half_year.mean().values
    sigma = half_year.cov().values

    dict_ports[port] = calc_ports(test, riskless_ret, index, mu, sigma)
    for date in dates[7:]:
        test = cluster[date]
        riskless_ret = bonds_monthly[date]
        index = sp500_monthly[date]
        
        half_year = pct_half_year[port][date]
        mu = half_year.mean().values
        sigma = half_year.cov().values
        layer = calc_ports(test, riskless_ret, index, mu, sigma)
        dict_ports[port] = pd.concat([dict_ports[port], layer])

  0%|          | 0/43 [00:00<?, ?it/s]

-0.18363962600439443 0.09553931418252
0.043144750674809086 0.08688130430105116
0.02024480553331665 0.17354599484816563
0.12043483796347262 0.19878889418294823
0.04275868211076441 0.0352428668891221
0.1465320409632577 0.022765905061804047
0.16905480134112683 -0.058288742143482654
0.12658363601633907 0.016789843905207106
0.4769098814347454 -0.006343624398066896
-0.016421078518027692 -0.034908505593700775
0.2653193292950516 0.03937372597749667
0.28609245132742317 -0.04061307861165393
0.08327090754429049 0.08045379676233158
-0.06498254941443851 -0.038408772938264425
0.11977043221265345 -0.04239297656060051
0.35780034193849364 0.015008100237215554
-0.16735054183118334 0.009081168716975742
-0.06080910646004153 0.10997293813768373
0.5288385881823139 0.1414213996622676
2.8027591976320534 0.10974530956125161
2.112080901188658 0.042158720105985856
0.655535107857522 -0.07011742237566214
-0.202218819049304 -0.0015676449740841653
0.11431228035423628 0.017160679620666575
0.1922307334213497 0.0219576

0.21927696756301848 -0.06546198834048708
0.03355018811228754 0.03692908032104825
-0.09970135943111504 -0.06621313456053893
0.03288858205182499 -0.1104896555875535
0.23295181780909455 -0.06624529948643605
-0.18896275104729313 -0.03400408017275982
-0.06439050615641495 0.08934691027696541
0.4494501640550565 0.10379805394402715
2.5750750337501187 0.10365728203375553
2.1399373461226916 0.06332011541160038
0.6906128557226812 -0.06264118421282087
-0.20580278466951007 0.002713354695460287
0.11942218970798457 0.02080328318127888
0.20262095548058373 0.025629680251864597
-0.16196798715327923 0.03850037615958591
0.4288018373492365 0.08771601527836335
0.40911306811929293 0.24478088853306848
0.9874649996738629 0.14951141401661613
-0.07811949586892158 0.18419656834788986
0.9133862511335679 0.22408388944055108
-0.745259483664883 0.1905055181479648
-0.4415426372808629 0.13624645719356465
1.0805316699973022 0.15233911280414145
0.8676932326939168 0.21880051585777338
-1.4725679308982373 0.1466220119293448

2.092726809192922 0.07485889540503764
0.6558665789439722 -0.06291928764940974
-0.19424420804200448 0.0074224492273159744
0.11852622838892601 0.025267971014346673
0.1940759135855665 0.026694101999485013
-0.14613655767215109 0.04348092180576694
0.4058804642826422 0.08442552166829721
0.39755431676627 0.24445859384297128
0.9388362998066037 0.16444329560428503
-0.048565804293754976 0.20499111721381516
0.8886906523268422 0.24574259755013894
-0.675896252660646 0.20559383036877485
-0.37652831181388224 0.17003849029773
1.0098105558871147 0.16353517036147905
0.8018846622316598 0.2082717604436877
-1.3679046344422905 0.12496569266090363
0.11165004690727967 0.12490395585953001
0.7019656246971768 0.12209168760032131
-1.1917501825616035 0.09071472565358156
-1.0749104175559616 0.10291811247903676
-0.1379420361322477 0.12024250233361229
-1.0817539866598076 0.08368488648409554
-1.566116520955883 0.04411236401387943
-0.31479905168808636 0.02993917017147127
-1.346465530043955 0.012054149355384102
-0.16577

-0.6835570018712309 0.20699146578882646
-0.392307326215217 0.16614148078763
1.028673419234991 0.15843716722200377
0.8136455500524222 0.20211327512061486
-1.4111485040270055 0.1295880683597002
0.11431797404614731 0.12920350251558976
0.7262049813973455 0.10730609898034862
-1.303127102889408 0.07067646066904455
-1.2102931877834688 0.0750585680285209
-0.18592853045828556 0.08958583946334976
-1.1796015633314008 0.04708581617209867
-1.7755861661399255 0.015710279302831774
-0.3575945831252039 0.027266058541747778
-1.51960863669828 0.009722693577452262
-0.1876674772494003 0.05185946994387932
-0.40404028748565507 -0.0031105172104901424
-0.5462540686841847 0.03916853445806648
-0.11958735150758175 -0.0009429572982988409
-0.21177851257874938 0.08641568156352497
-0.012478200588632507 0.042877679875023146
-0.08899222287636648 0.07014624912766354
-0.007715871374035333 0.07985366926107015
-0.008638638453578934 -0.007803473161053267
0.11977184571563106 0.0007801502203989343
0.1529961166158896 -0.074409

...

0.0934487311192094 0.10089290188550573
0.6495503386548606 0.11184542802365416
-1.0910762010960344 0.08612925894705585
-1.0203205499095698 0.08585440010038256
-0.14030232028904138 0.10441267145249629
-1.0650377467051568 0.07142471434586445
-1.5295487198632922 0.02885653429029379
-0.33861977992489717 -0.008137766192039582
-1.2877502694345566 -0.01461239681790881
-0.16551552539645353 0.028943299709950108
-0.3351990257373547 -0.017795375658490275
-0.4508228715319614 0.017885875873246738
-0.1200089595562634 -0.024467070603634112
-0.19997107016623114 0.05699971292168626
-0.02153664732940774 0.02747881826914554
-0.08164525793793714 0.06293562562299759
0.03995792694780901 0.12398851112260725
-0.0023837883951919945 0.0030757233396253615
0.0983278088993554 -0.0009566632048660629
0.1034050639850347 -0.09516036422474597
0.07860553428289582 -0.01390187737555115
0.42519300961185036 -0.004202033752711757
-0.020912000016835523 -0.0316338581470445
0.22888173409276502 0.02302017538588988
0.2253031936090

-0.0625593789326432 0.010988394790706173
-0.2535265631648987 0.03467236998015131
-0.03168620472041386 0.02428534065229893
-0.09538763023612065 0.0661575418940334
-0.013422989914828544 0.07897942079236926
-0.020316026463588155 -0.014439274218027431
0.1127680665225 -0.01040421826515416
0.17130019106514346 -0.06707109685310374
0.10464133744724625 -0.0038003565847392457
0.5004000738649288 0.003254269632723949
0.009133244734465303 -0.003388244832794603
0.2792207352249602 0.04396394632791243
0.3099962235640328 -0.01876102185034107
0.09650960873275709 0.10039587802395632
-0.0019487177243885383 0.033906412045264935
0.13249877576035454 -0.02041874458132601
0.3579951964602783 0.03668117465599383
-0.1058487919393023 0.06266621992747334
-0.013097356758741407 0.15566317921942496
0.5775638966483975 0.16938035566520515
2.981848628359224 0.12318859777945591
2.3624428601142626 0.08959934641796066
0.7513758046336966 -0.0679945415895971
-0.22731705065898175 -0.004850586345332553
0.12285810933510945 0.020

0.09029663665999796 0.006952429229797751
0.40746900790237844 0.02599516248423305
0.0023267471228276175 -0.0073220041017978054
0.25055357640964926 0.060539201418274846
0.22215397472418716 -0.037191489791573856
0.059934803519418385 0.06303743223035718
-0.07710618973266685 -0.04779450392459115
0.009203560302342094 -0.12152271427711268
0.20901724731802543 -0.06437321057272807
-0.18745155897768895 -0.04673210832528176
-0.08644961254916131 0.04975541091749701
0.41563926205426815 0.10033908265247217
2.311149465963077 0.10408950847471884
1.897067967055226 0.08532859236303557
0.5938877145092218 -0.06662202508010585
-0.17669926932451227 0.009679758065857916
0.11778932029666248 0.03113766035793958
0.18852084795683985 0.03286699776431556
-0.13654818410752637 0.04227977600626605
0.3835142170877838 0.08390281530853427
0.4058379697955123 0.2604261182681847
0.9057065523907182 0.16687283757236565
-0.02577496468949801 0.2125700836167211
0.8595635204197204 0.25515444784544034
-0.6003528321352273 0.232766

1.1463875154798417 0.20867257145956591
-0.0586725525006981 0.23134381174279212
0.9978126045924403 0.27566005000580285
-0.7020372261777751 0.22265842918886158
-0.40489909054046014 0.1697976421035093
1.029532084207336 0.15050090529209353
0.7998608130886532 0.19909985545946596
-1.2545065631354768 0.12586871934514982
0.11567205910661717 0.13183127689543594
0.6914714236937635 0.1253334384762198
-1.1559960028328078 0.11149477846386016
-1.0922134630633278 0.11889698586373142
-0.10941186845858918 0.15493749423759315
-1.11868204733816 0.09196413387254862
-1.6721575563973492 0.030416361425809398
-0.33845126279724913 0.007101928358116323
-1.360712388315942 -0.021259156043834857
-0.1814803612070439 0.025162806235440562
-0.3863468599671268 -0.0326514016934718
-0.5001219673845289 0.020576176259698514
-0.12154220830679359 -0.01524983931162851
-0.16083171280175912 0.07548399284243655
0.003442707238536853 0.049667275347805816
-0.06569358191568732 0.0654624907652459
0.05150500400910985 0.124424485272648

0.12113265306671106 0.1341246341082405
0.6622812665112257 0.1090881609602382
-1.1695298771196945 0.06703457701675682
-1.0401557279852987 0.08460063146541372
-0.1537257223110152 0.09096848217624075
-1.0703035689322964 0.01848467959734621
-1.5953443126301183 -0.00308713848899114
-0.3425043408501451 0.005565777220857055
-1.420369352530031 -0.013246987352322132
-0.20058937436230756 0.02091253376233072
-0.3822319524451446 -0.013300530055937967
-0.495923958563874 0.04671052029718468
-0.11605699697356923 -0.00936401355038886
-0.18363962600439443 0.09553931418252
0.043144750674809086 0.08688130430105116
0.02024480553331665 0.17354599484816563
0.12043483796347262 0.19878889418294823
0.04275868211076441 0.0352428668891221
0.1465320409632577 0.022765905061804047
0.16905480134112683 -0.058288742143482654
0.12658363601633907 0.016789843905207106
0.4769098814347454 -0.006343624398066896
-0.016421078518027692 -0.034908505593700775
0.2653193292950516 0.03937372597749667
0.28609245132742317 -0.04061307

In [21]:
df_res_sharp = pd.DataFrame([])
df_res_markov = pd.DataFrame([])
df_res_tobin = pd.DataFrame([])
columns = []

for i, (k, v) in enumerate(dict_ports.items()):
    col_names = v.columns
    df_loc_sharp = pd.DataFrame(data=v[col_names[1]].values, index=v[col_names[0]].values, columns = [k])
    df_res_sharp = pd.concat([df_res_sharp, df_loc_sharp], axis=1)
    df_loc_markov = pd.DataFrame(data=v[col_names[2]].values, index=v[col_names[0]].values, columns = [k])
    df_res_markov = pd.concat([df_res_markov, df_loc_markov], axis=1)
    df_loc_tobin = pd.DataFrame(data=v[col_names[3]].values, index=v[col_names[0]].values, columns = [k])
    df_res_tobin = pd.concat([df_res_tobin, df_loc_tobin], axis=1)
    columns.append(columns)

In [22]:
df_res_sharp['sp500']=sp500
df_res_markov['sp500']=sp500
df_res_tobin['sp500']=sp500

df_res_sharp.to_csv('results/df_res_sharp.csv')
df_res_markov.to_csv('results/df_res_markov.csv')
df_res_tobin.to_csv('results/df_res_tobin.csv')

In [23]:
df_real_sharp = (df_res_sharp + 1).cumprod()
df_real_markov = (df_res_markov + 1).cumprod()
df_real_tobin = (df_res_tobin + 1).cumprod()

df_real_sharp.to_csv('results/df_real_sharp.csv')
df_real_markov.to_csv('results/df_real_markov.csv')
df_real_tobin.to_csv('results/df_real_tobin.csv')

## Метрики Sharp

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import dates as mdates

from scipy.stats import norm

import functools

## Поиск метрик

In [25]:
df_all_ports_pct = pd.read_csv('results/df_res_sharp.csv', index_col=0)
df_all_ports_pct.index = pd.to_datetime(df_all_ports_pct.index)
days = df_all_ports_pct.index

## Доходность

In [26]:
ret_ports = df_all_ports_pct.mean()#находим доходности портфелей

## Безрисковый актив

In [27]:
df_b = pd.read_csv(r'data/bonds.csv', decimal=",")
df_bond = pd.DataFrame()

# Найти среднюю цену за месяц и поделить её на 100
df_bond['date'] = df_b['Дата']
df_bond['цена'] = df_b['Цена']/100
df_bond.date = pd.to_datetime(df_bond.date)
df_bond.set_index('date', inplace = True)

df_bond = df_bond.mean()/255

## Риск портфеля

In [28]:
risk_ports = df_all_ports_pct.std()

## Beta портфеля

In [29]:
var_ports = df_all_ports_pct.var()
cov_ports = df_all_ports_pct.cov()

In [30]:
# Beta = cov(ret_port;ret_IMOEX)/var(IMOEX), beta считается, только для портфелей, не для индекса
beta = cov_ports['sp500']/var_ports['sp500']
beta = beta.to_frame()
beta_ports = (cov_ports['sp500']/var_ports['sp500']).to_frame().T.rename({'sp500':'Beta'})

## Коэффициент VaR

In [31]:
mean_for_var = df_all_ports_pct.mean()
std_for_var = df_all_ports_pct.std()

In [32]:
l_pr_values = df_all_ports_pct.iloc[-1, :].values

In [33]:
var = l_pr_values*(1+norm.ppf(0.001, mean_for_var, std_for_var))/l_pr_values-1
var = pd.DataFrame(data=var.reshape(1,-1), columns=mean_for_var.index, index=['VaR'])
var

,original_n,deco_pca_Kmean,deco_pca_Agglo,deco_pca_MiniB,deco_pca_Gauss,deco_pca_Spect,deco_fast_Kmean,deco_fast_Agglo,deco_fast_MiniB,deco_fast_Gauss,...,neur_lstm_Gauss,neur_lstm_Spect,orig__Kmean,orig__Agglo,orig__MiniB,orig__Gauss,orig__Spect,orig__origi,orig__rando,sp500
VaR,-0.073099,-0.072941,-0.086899,-0.068909,-0.071966,-0.076654,-0.073532,-0.081765,-0.075231,-0.077019,...,-0.125835,-0.088104,-0.069313,-0.092476,-0.065519,-0.067204,-0.077554,-0.073099,-0.083227,-0.10566


## Коэффициент Шарпа

In [34]:
std_ports = df_all_ports_pct.std()

sharp_ports = (ret_ports - df_bond['цена'])/std_ports

## Просадка и восстановление

In [35]:
def find_max_drawdown(prices):

    max_price = prices.iloc[0]
    curr_drawdown = 0
    max_drawdown = 0
    curr_left = 0
    left = 0
    right = 0
    for i in range(0, len(prices)):
        curr_drawdown = (prices.iloc[i] / max_price - 1) * 100
        # print(type(curr_drawdown))
        if curr_drawdown < max_drawdown:
            max_drawdown = curr_drawdown
            left = curr_left
            right = i
        if prices.iloc[i] > max_price:
            max_price = prices.iloc[i]
            curr_left = i
    return max_drawdown

In [36]:
def find_max_recovery(prices):
    
    def calc_growth(prices):

        growth = []
        past_p = 0
        for p in prices:
            if past_p:
                growth.append(p - past_p)
            past_p = p
        return growth

    growth = calc_growth(prices)
    s = 0
    left = 0
    right = 0
    curr_left = 0
    max_recovery = 0
    for i in range(0, len(growth)):
        if not s:
            curr_left = i
        s += growth[i]
        if s > 0:
            s = 0
            if max_recovery < (i - curr_left):
                max_recovery = i - curr_left
                left = curr_left
                right = i

    return max_recovery

In [37]:
recovery = []
drawdown = []
for i in range(len(df_all_ports_pct.columns)):
    recovery.append(find_max_recovery(df_all_ports_pct.iloc[:,i]))
    drawdown.append(find_max_drawdown(df_all_ports_pct.iloc[:,i]))

In [38]:
risk = pd.DataFrame(risk_ports, columns =['Risk']).T
ret = pd.DataFrame(ret_ports, columns =['Ret']).T
sharp = pd.DataFrame(sharp_ports, columns =['Sharp']).T
rec = pd.Series(recovery)
rec = pd.DataFrame(rec, columns =['Recovery']).T
drawd = pd.Series(drawdown)
drawd = pd.DataFrame(drawd, columns =['Drawdown']).T

In [39]:
all_metrics = pd.concat([var,beta_ports, risk, ret, sharp])

rec.set_axis(list(all_metrics.columns), axis=1, inplace=True)
drawd.set_axis(list(all_metrics.columns), axis=1, inplace=True)

all_metrics = all_metrics.append(rec)
all_metrics = all_metrics.append(drawd)

all_metrics = all_metrics.T
all_metrics['Method'] = ['Sharp']*len(all_metrics)

## Метрики Markov

## Поиск метрик

In [40]:
df_all_ports_pct = pd.read_csv('results/df_res_markov.csv', index_col=0)
df_all_ports_pct.index = pd.to_datetime(df_all_ports_pct.index)
days = df_all_ports_pct.index

## Доходность

In [41]:
ret_ports = df_all_ports_pct.mean()#находим доходности портфелей

## Риск портфеля

In [42]:
risk_ports = df_all_ports_pct.std()

## Beta портфеля

In [43]:
var_ports = df_all_ports_pct.var()
cov_ports = df_all_ports_pct.cov()

In [44]:
# Beta = cov(ret_port;ret_IMOEX)/var(IMOEX), beta считается, только для портфелей, не для индекса
beta = cov_ports['sp500']/var_ports['sp500']
beta = beta.to_frame()
beta_ports = (cov_ports['sp500']/var_ports['sp500']).to_frame().T.rename({'sp500':'Beta'})

## Коэффициент VaR

In [45]:
mean_for_var = df_all_ports_pct.mean()
std_for_var = df_all_ports_pct.std()

In [46]:
l_pr_values = df_all_ports_pct.iloc[-1, :].values

In [47]:
var = l_pr_values*(1+norm.ppf(0.001, mean_for_var, std_for_var))/l_pr_values-1
var = pd.DataFrame(data=var.reshape(1,-1), columns=mean_for_var.index, index=['VaR'])
var

,original_n,deco_pca_Kmean,deco_pca_Agglo,deco_pca_MiniB,deco_pca_Gauss,deco_pca_Spect,deco_fast_Kmean,deco_fast_Agglo,deco_fast_MiniB,deco_fast_Gauss,...,neur_lstm_Gauss,neur_lstm_Spect,orig__Kmean,orig__Agglo,orig__MiniB,orig__Gauss,orig__Spect,orig__origi,orig__rando,sp500
VaR,-0.093972,-0.094132,-0.095374,-0.090891,-0.089692,-0.095817,-0.091084,-0.094908,-0.08804,-0.090707,...,-0.067222,-0.096279,-0.093674,-0.097429,-0.095736,-0.093042,-0.10031,-0.093972,-0.090263,-0.10566


## Коэффициент Шарпа

In [48]:
std_ports = df_all_ports_pct.std()
sharp_ports = (ret_ports - df_bond['цена'])/std_ports

## Просадка и восстановление

In [49]:
recovery = []
drawdown = []
for i in range(len(df_all_ports_pct.columns)):
    recovery.append(find_max_recovery(df_all_ports_pct.iloc[:,i]))
    drawdown.append(find_max_drawdown(df_all_ports_pct.iloc[:,i]))

In [50]:
risk = pd.DataFrame(risk_ports, columns =['Risk']).T
ret = pd.DataFrame(ret_ports, columns =['Ret']).T
sharp = pd.DataFrame(sharp_ports, columns =['Sharp']).T
rec = pd.Series(recovery)
rec = pd.DataFrame(rec, columns =['Recovery']).T
drawd = pd.Series(drawdown)
drawd = pd.DataFrame(drawd, columns =['Drawdown']).T

In [51]:
markov_metrics = pd.concat([var,beta_ports, risk, ret, sharp])

rec.set_axis(list(markov_metrics.columns), axis=1, inplace=True)
drawd.set_axis(list(markov_metrics.columns), axis=1, inplace=True)

markov_metrics = markov_metrics.append(rec)
markov_metrics = markov_metrics.append(drawd)

markov_metrics = markov_metrics.T
markov_metrics['Method'] = ['Markov']*len(all_metrics)

In [52]:
all_metrics = pd.concat([all_metrics, markov_metrics])

## Метрики Tobin

## Поиск метрик

In [53]:
df_all_ports_pct = pd.read_csv('results/df_res_tobin.csv', index_col=0)
df_all_ports_pct.index = pd.to_datetime(df_all_ports_pct.index)
days = df_all_ports_pct.index

## Доходность

In [54]:
ret_ports = df_all_ports_pct.mean()#находим доходности портфелей

## Риск портфеля

In [55]:
risk_ports = df_all_ports_pct.std()

## Beta портфеля

In [56]:
var_ports = df_all_ports_pct.var()
cov_ports = df_all_ports_pct.cov()

In [57]:
# Beta = cov(ret_port;ret_IMOEX)/var(IMOEX), beta считается, только для портфелей, не для индекса
beta = cov_ports['sp500']/var_ports['sp500']
beta = beta.to_frame()
beta_ports = (cov_ports['sp500']/var_ports['sp500']).to_frame().T.rename({'sp500':'Beta'})

## Коэффициент VaR

In [58]:
mean_for_var = df_all_ports_pct.mean()
std_for_var = df_all_ports_pct.std()

In [59]:
l_pr_values = df_all_ports_pct.iloc[-1, :].values

In [60]:
var = l_pr_values*(1+norm.ppf(0.001, mean_for_var, std_for_var))/l_pr_values-1
var = pd.DataFrame(data=var.reshape(1,-1), columns=mean_for_var.index, index=['VaR'])
var

,original_n,deco_pca_Kmean,deco_pca_Agglo,deco_pca_MiniB,deco_pca_Gauss,deco_pca_Spect,deco_fast_Kmean,deco_fast_Agglo,deco_fast_MiniB,deco_fast_Gauss,...,neur_lstm_Gauss,neur_lstm_Spect,orig__Kmean,orig__Agglo,orig__MiniB,orig__Gauss,orig__Spect,orig__origi,orig__rando,sp500
VaR,-0.108854,-0.118511,-0.118491,-0.122004,-0.122197,-0.118511,-0.118647,-0.113164,-0.120864,-0.118676,...,-0.118804,-0.118443,-0.118904,-0.113134,-0.110828,-0.110149,-0.116996,-0.108854,-0.112757,-0.10566


## Коэффициент Шарпа

In [61]:
std_ports = df_all_ports_pct.std()
sharp_ports = (ret_ports - df_bond['цена'])/std_ports

## Просадка и восстановление

In [62]:
recovery = []
drawdown = []
for i in range(len(df_all_ports_pct.columns)):
    recovery.append(find_max_recovery(df_all_ports_pct.iloc[:,i]))
    drawdown.append(find_max_drawdown(df_all_ports_pct.iloc[:,i]))

In [63]:
risk = pd.DataFrame(risk_ports, columns =['Risk']).T
ret = pd.DataFrame(ret_ports, columns =['Ret']).T
sharp = pd.DataFrame(sharp_ports, columns =['Sharp']).T
rec = pd.Series(recovery)
rec = pd.DataFrame(rec, columns =['Recovery']).T
drawd = pd.Series(drawdown)
drawd = pd.DataFrame(drawd, columns =['Drawdown']).T

In [64]:
tobin_metrics = pd.concat([var,beta_ports, risk, ret, sharp])

rec.set_axis(list(tobin_metrics.columns), axis=1, inplace=True)
drawd.set_axis(list(tobin_metrics.columns), axis=1, inplace=True)

tobin_metrics = tobin_metrics.append(rec)
tobin_metrics = tobin_metrics.append(drawd)

tobin_metrics = tobin_metrics.T
tobin_metrics['Method'] = ['Tobin']*len(tobin_metrics)

In [65]:
all_metrics = pd.concat([all_metrics, tobin_metrics])

## Сохранение результата

In [66]:
all_metrics.to_excel('results/metrics.xlsx')